In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
from datetime import datetime
import calendar

class Caregiver:
    def __init__(self, name: str, phone: str, email: str, is_paid: bool = False):
        self.name = name
        self.phone = phone
        self.email = email
        self.is_paid = is_paid
        self.pay_rate = 20.0 if is_paid else 0.0
        self.hours = 0
        self.availability = self._create_default_availability()
        
    def _create_default_availability(self) -> dict:
        """Creates default weekly availability schedule"""
        schedule = {}
        for day in range(7):  # 0 = Monday, 6 = Sunday
            schedule[day] = {
                'AM': 'available',  # 7:00 AM - 1:00 PM
                'PM': 'available'   # 1:00 PM - 7:00 PM
            }
        return schedule
    
    def set_availability(self, day: int, shift: str, status: str):
        """Set availability for a specific day and shift"""
        if status not in ['preferred', 'available', 'unavailable']:
            raise ValueError("Status must be 'preferred', 'available', or 'unavailable'")
        self.availability[day][shift] = status
        
    def add_hours(self, hours: float):
        """Add worked hours"""
        self.hours += hours

class Schedule:
    def __init__(self, year: int, month: int):
        self.year = year
        self.month = month
        self.caregivers = []
        self.schedule = {}
        
    def add_caregiver(self, caregiver: Caregiver):
        self.caregivers.append(caregiver)
        
    def generate_schedule(self):
        """Generate monthly schedule based on caregiver availability"""
        _, num_days = calendar.monthrange(self.year, self.month)
        
        for day in range(1, num_days + 1):
            weekday = calendar.weekday(self.year, self.month, day)
            
            # Find available caregivers for each shift
            am_shift = self._assign_shift(weekday, 'AM')
            pm_shift = self._assign_shift(weekday, 'PM')
            
            self.schedule[day] = {
                'AM': am_shift,
                'PM': pm_shift
            }
            
            # Add hours for paid caregivers
            if am_shift and am_shift.is_paid:
                am_shift.add_hours(6)
            if pm_shift and pm_shift.is_paid:
                pm_shift.add_hours(6)
    
    def _assign_shift(self, weekday: int, shift: str) -> Caregiver:
        """Assign a caregiver to a shift based on availability"""
        preferred = [c for c in self.caregivers if c.availability[weekday][shift] == 'preferred']
        available = [c for c in self.caregivers if c.availability[weekday][shift] == 'available']
        
        if preferred:
            return min(preferred, key=lambda x: x.hours)
        elif available:
            return min(available, key=lambda x: x.hours)
        return None

    def generate_html_calendar(self):
        """Generate HTML calendar with schedule"""
        html = f"""
        <html>
        <head>
            <title>Care Schedule - {calendar.month_name[self.month]} {self.year}</title>
            <style>
                table {{ border-collapse: collapse; width: 100%; }}
                th, td {{ border: 1px solid black; padding: 10px; text-align: center; }}
                th {{ background-color: #f2f2f2; }}
            </style>
        </head>
        <body>
            <h1>Care Schedule - {calendar.month_name[self.month]} {self.year}</h1>
            <table>
                <tr>
                    <th>Mon</th><th>Tue</th><th>Wed</th><th>Thu</th><th>Fri</th><th>Sat</th><th>Sun</th>
                </tr>
        """
        
        first_day = calendar.weekday(self.year, self.month, 1)
        _, num_days = calendar.monthrange(self.year, self.month)
        
        current_day = 1
        html += "<tr>"
        
        for _ in range(first_day):
            html += "<td></td>"
            
        while current_day <= num_days:
            if (first_day + current_day - 1) % 7 == 0 and current_day != 1:
                html += "</tr><tr>"
                
            day_schedule = self.schedule.get(current_day, {'AM': None, 'PM': None})
            am_name = day_schedule['AM'].name if day_schedule['AM'] else "OPEN"
            pm_name = day_schedule['PM'].name if day_schedule['PM'] else "OPEN"
            
            html += f"""
                <td>
                    {current_day}<br>
                    <b>AM:</b> {am_name}<br>
                    <b>PM:</b> {pm_name}
                </td>
            """
            
            current_day += 1
            
        while (first_day + current_day - 1) % 7 != 0:
            html += "<td></td>"
            current_day += 1
            
        html += """
                </tr>
            </table>
        </body>
        </html>
        """
        
        with open(f"care_schedule_{self.year}_{self.month}.html", "w") as f:
            f.write(html)

def generate_pay_report(schedule: Schedule):
    """Generate weekly pay report for paid caregivers"""
    report = f"Pay Report for {calendar.month_name[schedule.month]} {schedule.year}\n"
    report += "=" * 50 + "\n\n"
    
    paid_caregivers = [c for c in schedule.caregivers if c.is_paid]
    total_monthly_pay = 0
    
    for caregiver in paid_caregivers:
        weekly_hours = caregiver.hours / 4  # Approximate weekly hours
        weekly_pay = weekly_hours * caregiver.pay_rate
        monthly_pay = weekly_pay * 4
        
        report += f"Caregiver: {caregiver.name}\n"
        report += f"Weekly Hours: {weekly_hours:.2f}\n"
        report += f"Weekly Pay: ${weekly_pay:.2f}\n"
        report += f"Monthly Pay: ${monthly_pay:.2f}\n"
        report += "-" * 30 + "\n"
        
        total_monthly_pay += monthly_pay
    
    report += f"\nTotal Monthly Payroll: ${total_monthly_pay:.2f}"
    
    with open(f"pay_report_{schedule.month}_{schedule.year}.txt", "w") as f:
        f.write(report)

class CareSchedulerGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Care Schedule Generator")
        self.root.geometry("800x600")
        
        # Initialize caregivers list first
        self.caregivers = [
            Caregiver("Alice Smith", "555-0101", "alice@email.com", True),
            Caregiver("Bob Jones", "555-0102", "bob@email.com", True),
            Caregiver("Carol Wilson", "555-0103", "carol@email.com", False),
            Caregiver("David Brown", "555-0104", "david@email.com", True),
            Caregiver("Eve Davis", "555-0105", "eve@email.com", False),
            Caregiver("Frank Miller", "555-0106", "frank@email.com", True),
            Caregiver("Grace Taylor", "555-0107", "grace@email.com", False),
            Caregiver("Henry White", "555-0108", "henry@email.com", True)
        ]
        
        # Store caregiver availability data
        self.availability_data = {}
        
        # Create main containers
        self.create_header_frame()
        self.create_date_frame()
        self.create_caregiver_frame()
        self.create_buttons_frame()

    def create_header_frame(self):
        """Create the header section"""
        header_frame = ttk.Frame(self.root, padding="10")
        header_frame.grid(row=0, column=0, sticky=(tk.W, tk.E))
        
        ttk.Label(header_frame, 
                 text="Care Schedule Generator", 
                 font=('Helvetica', 16, 'bold')).grid(row=0, column=0)

    def create_date_frame(self):
        """Create the date selection section"""
        date_frame = ttk.LabelFrame(self.root, text="Schedule Period", padding="10")
        date_frame.grid(row=1, column=0, padx=10, pady=5, sticky=(tk.W, tk.E))
        
        # Year selection
        current_year = datetime.now().year
        ttk.Label(date_frame, text="Year:").grid(row=0, column=0, padx=5)
        self.year_var = tk.StringVar(value=str(current_year))
        year_combo = ttk.Combobox(date_frame, 
                                 textvariable=self.year_var, 
                                 values=[str(current_year), str(current_year + 1)],
                                 width=10,
                                 state='readonly')
        year_combo.grid(row=0, column=1, padx=5)
        
        # Month selection
        ttk.Label(date_frame, text="Month:").grid(row=0, column=2, padx=5)
        self.month_var = tk.StringVar(value=calendar.month_name[datetime.now().month])
        month_combo = ttk.Combobox(date_frame, 
                                  textvariable=self.month_var,
                                  values=[calendar.month_name[i] for i in range(1, 13)],
                                  width=10,
                                  state='readonly')
        month_combo.grid(row=0, column=3, padx=5)

    def create_caregiver_frame(self):
        """Create the caregiver availability section"""
        caregiver_frame = ttk.LabelFrame(self.root, text="Caregiver Availability", padding="10")
        caregiver_frame.grid(row=2, column=0, padx=10, pady=5, sticky=(tk.W, tk.E))
        
        # Create notebook for days of the week
        self.notebook = ttk.Notebook(caregiver_frame)
        self.notebook.grid(row=0, column=0, sticky=(tk.W, tk.E))
        
        # Create a tab for each day
        self.day_frames = {}
        for day in calendar.day_name:
            frame = ttk.Frame(self.notebook, padding="10")
            self.notebook.add(frame, text=day)
            self.day_frames[day] = frame
            self.create_day_schedule(frame, day)

    def create_day_schedule(self, frame, day):
        """Create the schedule grid for a single day"""
        # Headers
        ttk.Label(frame, text="Caregiver", font=('Helvetica', 10, 'bold')).grid(row=0, column=0, padx=5)
        ttk.Label(frame, text="Morning (7AM-1PM)", font=('Helvetica', 10, 'bold')).grid(row=0, column=1, padx=5)
        ttk.Label(frame, text="Afternoon (1PM-7PM)", font=('Helvetica', 10, 'bold')).grid(row=0, column=2, padx=5)
        
        # Create entries for each caregiver
        for i, caregiver in enumerate(self.caregivers, 1):
            ttk.Label(frame, text=caregiver.name).grid(row=i, column=0, padx=5, pady=2)
            
            # Morning availability
            am_var = tk.StringVar(value='available')
            am_combo = ttk.Combobox(frame, 
                                  textvariable=am_var,
                                  values=['preferred', 'available', 'unavailable'],
                                  width=15,
                                  state='readonly')
            am_combo.grid(row=i, column=1, padx=5, pady=2)
            
            # Afternoon availability
            pm_var = tk.StringVar(value='available')
            pm_combo = ttk.Combobox(frame, 
                                  textvariable=pm_var,
                                  values=['preferred', 'available', 'unavailable'],
                                  width=15,
                                  state='readonly')
            pm_combo.grid(row=i, column=2, padx=5, pady=2)
            
            # Store the variables
            key = (day, caregiver.name)
            self.availability_data[key] = {'AM': am_var, 'PM': pm_var}

    def create_buttons_frame(self):
        """Create the buttons section"""
        buttons_frame = ttk.Frame(self.root, padding="10")
        buttons_frame.grid(row=3, column=0, pady=20, sticky=(tk.W, tk.E))
        
        ttk.Button(buttons_frame, 
                  text="Generate Schedule",
                  command=self.generate_schedule).grid(row=0, column=0, padx=5)
        
        ttk.Button(buttons_frame,
                  text="Exit",
                  command=self.root.quit).grid(row=0, column=1, padx=5)

    def generate_schedule(self):
        """Generate the schedule based on input"""
        try:
            # Get selected year and month
            year = int(self.year_var.get())
            month = list(calendar.month_name).index(self.month_var.get())
            
            # Update caregiver availability based on GUI input
            for caregiver in self.caregivers:
                for day_idx, day in enumerate(calendar.day_name):
                    key = (day, caregiver.name)
                    if key in self.availability_data:
                        am_status = self.availability_data[key]['AM'].get()
                        pm_status = self.availability_data[key]['PM'].get()
                        caregiver.set_availability(day_idx, 'AM', am_status)
                        caregiver.set_availability(day_idx, 'PM', pm_status)
            
            # Create and generate schedule
            schedule = Schedule(year, month)
            for caregiver in self.caregivers:
                schedule.add_caregiver(caregiver)
            
            schedule.generate_schedule()
            schedule.generate_html_calendar()
            generate_pay_report(schedule)
            
            messagebox.showinfo("Success", 
                              f"Schedule generated successfully for {calendar.month_name[month]} {year}!\n\n"
                              f"Files created:\n"
                              f"- care_schedule_{year}_{month}.html\n"
                              f"- pay_report_{month}_{year}.txt")
            
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {str(e)}")

def main():
    root = tk.Tk()
    app = CareSchedulerGUI(root)
    root.mainloop()

if __name__ == "__main__":
    main()